
# Exploration interactive du dataset 🐧*Penguins*🐧 avec **ipywidgets**

Nous allons utiliser le dataset Penguins et la bibliothèque [ipywidgets](https://ipywidgets.readthedocs.io/)  
pour créer des visualisations interactives.


In [1]:
# Les imports
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive_output, HBox, VBox
from IPython.display import display

In [2]:
# Charger le dataset
penguins = sns.load_dataset("penguins")
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


## 1. Choisir une espèce avec un menu déroulant

In [3]:
species_dropdown = widgets.Dropdown(
                                    options=penguins['species'].dropna().unique().tolist(),
                                    value='Adelie',
                                    description='Espèce:'
                                    )

def afficher_taille(species):
    subset = penguins[penguins['species'] == species]
    plt.figure(figsize=(6,4))
    plt.hist(subset['body_mass_g'].dropna(), bins=15, color="skyblue", edgecolor="black")
    plt.xlabel("Masse corporelle (g)")
    plt.ylabel("Nombre de pingouins")
    plt.title(f"Masse corporelle des {species}")
    plt.show()

interact(afficher_taille, species=species_dropdown);

interactive(children=(Dropdown(description='Espèce:', options=('Adelie', 'Chinstrap', 'Gentoo'), value='Adelie…

## 2. Filtrer par masse corporelle avec un slider

In [4]:
mass_slider = widgets.IntSlider(
                                value=4000,
                                min=2500,
                                max=6500,
                                step=100,
                                description='Masse min:'
                                )

def filtre_masse(masse_min):
    subset = penguins[penguins['body_mass_g'] >= masse_min]
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=subset, x="flipper_length_mm", y="body_mass_g", hue="species")
    plt.title(f"Pingouins avec masse >= {masse_min} g")
    plt.show()

interact(filtre_masse, masse_min=mass_slider);

interactive(children=(IntSlider(value=4000, description='Masse min:', max=6500, min=2500, step=100), Output())…

## 3. Choisir une variable à visualiser avec des boutons radio

In [5]:
var_radio = widgets.RadioButtons(
                                options=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
                                description='Variable:'
                                )

def plot_variable(var):
    plt.figure(figsize=(6,4))
    sns.boxplot(data=penguins, x="species", y=var)
    plt.title(f"Distribution de {var} par espèce")
    plt.show()

interact(plot_variable, var=var_radio);

interactive(children=(RadioButtons(description='Variable:', options=('bill_length_mm', 'bill_depth_mm', 'flipp…

## 4. Combiner plusieurs widgets

In [6]:
species_dd = widgets.Dropdown(options=penguins['species'].dropna().unique().tolist(), description="Espèce:")
sex_dd = widgets.Dropdown(options=penguins['sex'].dropna().unique().tolist(), description="Sexe:")
var_dd = widgets.Dropdown(options=['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g'], description="Variable:")

def afficher_distribution(species, sex, var):
    subset = penguins[(penguins['species']==species) & (penguins['sex']==sex)]
    plt.figure(figsize=(6,4))
    plt.hist(subset[var].dropna(), bins=10, color="lightgreen", edgecolor="black")
    plt.title(f"Distribution de {var} chez {species} ({sex})")
    plt.xlabel(var)
    plt.ylabel("Effectif")
    plt.show()

ui = VBox([species_dd, sex_dd, var_dd])
out = interactive_output(afficher_distribution, {'species': species_dd, 'sex': sex_dd, 'var': var_dd})

display(ui, out)

Output()

## Pourquoi choisir quand on peut tout mettre ? 😁

In [7]:
species_dd = widgets.Dropdown(
                            options=['Toutes'] + penguins['species'].dropna().unique().tolist(),
                            value='Toutes',
                            description="Espèce:"
                            )

sex_dd = widgets.Dropdown(
                        options=['Tous'] + penguins['sex'].dropna().unique().tolist(),
                        value='Tous',
                        description="Sexe:"
                        )

x_dd = widgets.Dropdown(
                        options=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
                        value='bill_length_mm',
                        description="Axe X:"
                        )

y_dd = widgets.Dropdown(
                        options=['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'],
                        value='flipper_length_mm',
                        description="Axe Y:"
                        )

mass_slider = widgets.IntSlider(
                                value=3000,
                                min=2500,
                                max=6500,
                                step=100,
                                description="Masse min:"
                                )

plot_type = widgets.RadioButtons(
                                options=['scatter', 'boxplot', 'hist'],
                                value='scatter',
                                description="Graphique:"
                                )


def visualisation(species, sex, x, y, masse_min, plot_type):
    # Filtrage du dataset
    subset = penguins.dropna()
    if species != "Toutes":
        subset = subset[subset['species'] == species]
    if sex != "Tous":
        subset = subset[subset['sex'] == sex]
    subset = subset[subset['body_mass_g'] >= masse_min]

    # Création du graphique
    plt.figure(figsize=(7,5))

    if plot_type == "scatter":
        sns.scatterplot(data=subset, x=x, y=y, hue="species")
    elif plot_type == "boxplot":
        sns.boxplot(data=subset, x="species", y=y, hue="sex")
    elif plot_type == "hist":
        sns.histplot(data=subset, x=x, hue="species", multiple="stack", bins=15)

    plt.title(f"Visualisation : {plot_type}")
    plt.show()


# Interface combinée
ui = VBox([
        HBox([species_dd, sex_dd]),
        HBox([x_dd, y_dd]),
        HBox([mass_slider, plot_type])
        ])

out = interactive_output(
                        visualisation,
                        {'species': species_dd, 'sex': sex_dd, 'x': x_dd, 'y': y_dd, 'masse_min': mass_slider, 'plot_type': plot_type}
                        )

display(out, ui)

Output()